In [1]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Utils.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
from Utils import Reader
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
#ALL TRIES
import numpy
import itertools
from tqdm.notebook import tqdm
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
# GRID SEARCH RP3BETA

URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
#URM_all=Reader.load_URM("../output/URM_2_mean.csv")
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all ,train_percentage=0.80)
evaluator_test=EvaluatorHoldout(URM_test,cutoff_list=[10])

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
EvaluatorHoldout: Ignoring 328 ( 0.8%) Users that have less than 1 test interactions


In [2]:
slimBPR=SLIM_BPR_Cython(URM_train)
slimBPR.fit()
result_df, _ = evaluator_test.evaluateRecommender(slimBPR)
result_df

/bin/sh: free: command not found


Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 1.50 sec. BPR loss is 1.72E-03. Sample per second: 27818
SLIM_BPR_Recommender: Epoch 1 of 420. Elapsed time 1.15 sec
Processed 41629 (100.0%) in 1.01 sec. BPR loss is 5.17E-03. Sample per second: 41089
SLIM_BPR_Recommender: Epoch 2 of 420. Elapsed time 1.60 sec
Processed 41629 (100.0%) in 0.29 sec. BPR loss is 8.48E-03. Sample per second: 141510
SLIM_BPR_Recommender: Epoch 3 of 420. Elapsed time 1.88 sec
Processed 41629 (100.0%) in 0.53 sec. BPR loss is 1.28E-02. Sample per second: 77988
SLIM_BPR_Recommender: Epoch 4 of 420. Elapsed time 2.11 sec
Processed 41629 (100.0%) in 0.74 sec. BPR loss is 1.50E-02. Sample per second: 55867
SLIM_BPR_Recommender: Epoch 5 of 420. Elapsed time 2.33 sec
Processed 41629 (100.0%) in 0.98 sec. BPR loss is 1.93

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 41301 (100.0%) in 40.16 sec. Users per second: 1028


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.050488,0.070592,0.062195,0.024732,0.0342,0.167998,NaN,0.055733,0.339822,0.202311,...,0.992121,0.337145,0.992121,0.020329,7.418021,0.979068,0.040056,0.549741,6.359703,0.240796


In [5]:

ials=IALSRecommender(URM_train)
ials.fit( num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
ItemKNNCF = ItemKNNCFRecommender(URM_train)
ItemKNNCF.fit()
rp3=RP3betaRecommender(URM_train)
rp3.fit(
    topK= 339, alpha= 0.0, beta= 0.3326953675715214, normalize_similarity= True
)
pureSVD=PureSVDRecommender(URM_train)
pureSVD.fit(
    num_factors= 30
)

P3alpha=P3alphaRecommender(URM_train)
P3alpha.fit(
    topK= 564, alpha= 0.18769516845528106, normalize_similarity= True
)

slimBPR=SLIM_BPR_Cython(URM_train)
slimBPR.fit(

)

IALSRecommender: Epoch 1 of 115. Elapsed time 1.19 min
IALSRecommender: Epoch 2 of 115. Elapsed time 1.77 min
IALSRecommender: Epoch 3 of 115. Elapsed time 2.32 min
IALSRecommender: Epoch 4 of 115. Elapsed time 2.85 min
IALSRecommender: Epoch 5 of 115. Elapsed time 3.49 min
IALSRecommender: Epoch 6 of 115. Elapsed time 4.05 min
IALSRecommender: Epoch 7 of 115. Elapsed time 4.61 min
IALSRecommender: Epoch 8 of 115. Elapsed time 5.15 min
IALSRecommender: Epoch 9 of 115. Elapsed time 5.77 min
IALSRecommender: Epoch 10 of 115. Elapsed time 6.35 min
IALSRecommender: Epoch 11 of 115. Elapsed time 6.93 min
IALSRecommender: Epoch 12 of 115. Elapsed time 7.51 min
IALSRecommender: Epoch 13 of 115. Elapsed time 8.18 min
IALSRecommender: Epoch 14 of 115. Elapsed time 8.77 min
IALSRecommender: Epoch 15 of 115. Elapsed time 9.31 min
IALSRecommender: Epoch 16 of 115. Elapsed time 9.85 min
IALSRecommender: Epoch 17 of 115. Elapsed time 10.49 min
IALSRecommender: Epoch 18 of 115. Elapsed time 11.07 min

/bin/sh: free: command not found


Processed 41629 (100.0%) in 3.86 sec. BPR loss is 1.85E-02. Sample per second: 10781
SLIM_BPR_Recommender: Epoch 1 of 140. Elapsed time 3.95 sec
Processed 41629 (100.0%) in 3.22 sec. BPR loss is 5.42E-02. Sample per second: 12939
SLIM_BPR_Recommender: Epoch 2 of 140. Elapsed time 6.98 sec
Processed 41629 (100.0%) in 1.17 sec. BPR loss is 8.49E-02. Sample per second: 35484
SLIM_BPR_Recommender: Epoch 3 of 140. Elapsed time 7.93 sec
Processed 41629 (100.0%) in 1.89 sec. BPR loss is 1.12E-01. Sample per second: 21818
SLIM_BPR_Recommender: Epoch 4 of 140. Elapsed time 9.68 sec
Processed 41629 (100.0%) in 4.88 sec. BPR loss is 1.48E-01. Sample per second: 8522
SLIM_BPR_Recommender: Epoch 5 of 140. Elapsed time 13.66 sec
Processed 41629 (100.0%) in 4.53 sec. BPR loss is 1.76E-01. Sample per second: 9168
SLIM_BPR_Recommender: Epoch 6 of 140. Elapsed time 17.30 sec
Processed 41629 (100.0%) in 4.17 sec. BPR loss is 1.99E-01. Sample per second: 9975
SLIM_BPR_Recommender: Epoch 7 of 140. Elapsed 

In [8]:
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
ials=IALSRecommender(URM_all)
ials.fit( num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
ItemKNNCF = ItemKNNCFRecommender(URM_all)
ItemKNNCF.fit()
alpha=0.40
beta=0.40
slimBPR=SLIM_BPR_Cython(URM_all)
slimBPR.fit(

)
higher_recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_all,
    recommenders=[
        ItemKNNCF,
        ials,
        slimBPR

    ],
    verbose=False
)
higher_recommender.fit(
    alphas=[
        alpha,beta,
        1 - alpha - beta
    ]
)
import os
import numpy as np
import csv
import pandas as pd
parent_dir =  "../"
read_path = os.path.join(parent_dir, "data/data_target_users_test.csv")

with open(read_path, "r") as f:
    dataFrame = pd.read_csv(f, skiprows=0)
    dataFrame.columns = ["UserID"]
    user_array = np.asarray(dataFrame["UserID"])

write_path = os.path.join(parent_dir, "Testing_Results/submission3.csv")
print(user_array)
user_array = user_array.tolist()
with open(write_path, "w+") as d:
    d.write("user_id,item_list\n")
    writer = csv.writer(d, delimiter=' ')

    for i in range(len(user_array)):
        d.write(str(user_array[i]) + ", ")
        #             if(self.NameRecommender.name!="Rankings"):
        recommandations = higher_recommender.recommend(user_id_array=user_array[i], cutoff=10)
        #               else:
        #                    recommandations=self.Recommender.recomendation_ranking(user_id_array=user_array[i])
        array = np.asarray(recommandations)
        writer.writerow(array)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
IALSRecommender: Epoch 1 of 115. Elapsed time 31.58 sec
IALSRecommender: Epoch 2 of 115. Elapsed time 49.38 sec
IALSRecommender: Epoch 3 of 115. Elapsed time 1.11 min
IALSRecommender: Epoch 4 of 115. Elapsed time 1.43 min
IALSRecommender: Epoch 5 of 115. Elapsed time 1.72 min
IALSRecommender: Epoch 6 of 115. Elapsed time 2.01 min
IALSRecommender: Epoch 7 of 115. Elapsed time 2.50 min
IALSRecommender: Epoch 8 of 115. Elapsed time 2.91 min
IALSRecommender: Epoch 9 of 115. Elapsed time 3.19 min
IALSRecommender: Epoch 10 of 115. Elapsed time 3.48 min
IALSRecommender: Epoch 11 of 115. Elapsed time 3.87 min
IALSRecommender: Epoch 12 of 115. Elapsed time 4.15 min
IALSRecommender: Epoch 13 of 115. Elapsed time 4.43 min
IALSRecommender: Epoch 14 of 115. Elapsed time 4.71 min
IALSRecommender: Epoch 15 of 115. Elapsed time 5.00 min
IALSRecommender: Epoch 16 of 115. Elapsed time 5.28 min
IALSRecommender: Epoch 17

/bin/sh: free: command not found


Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 1.21 sec. BPR loss is 3.92E-02. Sample per second: 34445
SLIM_BPR_Recommender: Epoch 1 of 140. Elapsed time 0.84 sec
Processed 41629 (100.0%) in 0.57 sec. BPR loss is 1.17E-01. Sample per second: 73407
SLIM_BPR_Recommender: Epoch 2 of 140. Elapsed time 1.19 sec
Processed 41629 (100.0%) in 0.79 sec. BPR loss is 1.74E-01. Sample per second: 52962
SLIM_BPR_Recommender: Epoch 3 of 140. Elapsed time 1.41 sec
Processed 41629 (100.0%) in 1.06 sec. BPR loss is 2.32E-01. Sample per second: 39263
SLIM_BPR_Recommender: Epoch 4 of 140. Elapsed time 1.69 sec
Processed 41629 (100.0%) in 0.26 sec. BPR loss is 2.85E-01. Sample per second: 158128
SLIM_BPR_Recommender: Epoch 5 of 140. Elapsed time 1.89 sec
Processed 41629 (100.0%) in 0.50 sec. BPR loss is 3.28

In [9]:
import os
import numpy as np
import csv
import pandas as pd
parent_dir =  "../"
read_path = os.path.join(parent_dir, "data/data_target_users_test.csv")

with open(read_path, "r") as f:
    dataFrame = pd.read_csv(f, skiprows=0)
    dataFrame.columns = ["UserID"]
    user_array = np.asarray(dataFrame["UserID"])

write_path = os.path.join(parent_dir, "Testing_Results/submission3.csv")
print(user_array)
user_array = user_array.tolist()
with open(write_path, "w+") as d:
    d.write("user_id,item_list\n")
    writer = csv.writer(d, delimiter=' ')

    for i in range(len(user_array)):
        d.write(str(user_array[i]) + ", ")
        #             if(self.NameRecommender.name!="Rankings"):
        recommandations = higher_recommender.recommend(user_id_array=user_array[i], cutoff=10)
        #               else:
        #                    recommandations=self.Recommender.recomendation_ranking(user_id_array=user_array[i])
        array = np.asarray(recommandations)
        writer.writerow(array)

[    0     1     2 ... 41626 41627 41628]


In [11]:
ials=IALSRecommender(URM_train)
ials.fit( num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
ItemKNNCF = ItemKNNCFRecommender(URM_train)
ItemKNNCF.fit()
slimBPR=SLIM_BPR_Cython(URM_train)
slimBPR.fit(

)

IALSRecommender: Epoch 1 of 115. Elapsed time 26.92 sec
IALSRecommender: Epoch 2 of 115. Elapsed time 44.66 sec
IALSRecommender: Epoch 3 of 115. Elapsed time 1.12 min
IALSRecommender: Epoch 4 of 115. Elapsed time 1.57 min
IALSRecommender: Epoch 5 of 115. Elapsed time 1.92 min
IALSRecommender: Epoch 6 of 115. Elapsed time 2.29 min
IALSRecommender: Epoch 7 of 115. Elapsed time 2.78 min
IALSRecommender: Epoch 8 of 115. Elapsed time 3.04 min
IALSRecommender: Epoch 9 of 115. Elapsed time 3.29 min
IALSRecommender: Epoch 10 of 115. Elapsed time 3.54 min
IALSRecommender: Epoch 11 of 115. Elapsed time 3.79 min
IALSRecommender: Epoch 12 of 115. Elapsed time 4.04 min
IALSRecommender: Epoch 13 of 115. Elapsed time 4.29 min
IALSRecommender: Epoch 14 of 115. Elapsed time 4.54 min
IALSRecommender: Epoch 15 of 115. Elapsed time 4.78 min
IALSRecommender: Epoch 16 of 115. Elapsed time 5.03 min
IALSRecommender: Epoch 17 of 115. Elapsed time 5.28 min
IALSRecommender: Epoch 18 of 115. Elapsed time 5.52 min

/bin/sh: free: command not found


Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 2.09 sec. BPR loss is 1.87E-02. Sample per second: 19883
SLIM_BPR_Recommender: Epoch 1 of 140. Elapsed time 1.15 sec
Processed 41629 (100.0%) in 0.90 sec. BPR loss is 5.39E-02. Sample per second: 46197
SLIM_BPR_Recommender: Epoch 2 of 140. Elapsed time 1.94 sec
Processed 41629 (100.0%) in 1.18 sec. BPR loss is 8.73E-02. Sample per second: 35230
SLIM_BPR_Recommender: Epoch 3 of 140. Elapsed time 2.22 sec
Processed 41629 (100.0%) in 0.41 sec. BPR loss is 1.10E-01. Sample per second: 102524
SLIM_BPR_Recommender: Epoch 4 of 140. Elapsed time 2.45 sec
Processed 41629 (100.0%) in 0.59 sec. BPR loss is 1.46E-01. Sample per second: 70031
SLIM_BPR_Recommender: Epoch 5 of 140. Elapsed time 2.63 sec
Processed 41629 (100.0%) in 0.75 sec. BPR loss is 1.70

In [12]:
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender

beta=0
best=0
alpha=0
algo=""
dList=numpy.arange(0.25, 0.5, 0.1)
algoList=numpy.arange(0.25, 0.5, 0.25)

for x in tqdm(itertools.product(dList,algoList),total=int(len(dList)*len(algoList)), position=0, leave=True):

    higher_recommender = GeneralizedMergedHybridRecommender(
        URM_train=URM_train,
        recommenders=[
            ItemKNNCF,
            ials,
            slimBPR

        ],
        verbose=False
    )
    higher_recommender.fit(
        alphas=[
            x[0],x[1],
            1 - x[0] - x[1]
        ]
    )

    result_df, _ = evaluator_test.evaluateRecommender(higher_recommender)
    result_df
    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        alpha=x[0]
        beta=x[1]


  0%|          | 0/3 [00:00<?, ?it/s]

Deallocating Cython objects


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 41326 (100.0%) in 1.21 min. Users per second: 571


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 41326 (100.0%) in 1.31 min. Users per second: 525


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 41326 (100.0%) in 1.23 min. Users per second: 562


In [14]:
alpha,beta

(0.44999999999999996, 0.25)

In [17]:
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
ials=IALSRecommender(URM_all)
alpha=0.50
beta=0.25
ials.fit( num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
ItemKNNCF = ItemKNNCFRecommender(URM_all)
ItemKNNCF.fit()

slimBPR=SLIM_BPR_Cython(URM_all)
slimBPR.fit(

)
higher_recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_all,
    recommenders=[
        ItemKNNCF,
        ials,
        slimBPR

    ],
    verbose=False
)
higher_recommender.fit(
    alphas=[
        alpha,beta,
        1 - alpha - beta
    ]
)
import os
import numpy as np
import csv
import pandas as pd
parent_dir =  "../"
read_path = os.path.join(parent_dir, "data/data_target_users_test.csv")

with open(read_path, "r") as f:
    dataFrame = pd.read_csv(f, skiprows=0)
    dataFrame.columns = ["UserID"]
    user_array = np.asarray(dataFrame["UserID"])

write_path = os.path.join(parent_dir, "Testing_Results/submission4.csv")
print(user_array)
user_array = user_array.tolist()
with open(write_path, "w+") as d:
    d.write("user_id,item_list\n")
    writer = csv.writer(d, delimiter=' ')

    for i in range(len(user_array)):
        d.write(str(user_array[i]) + ", ")
        #             if(self.NameRecommender.name!="Rankings"):
        recommandations = higher_recommender.recommend(user_id_array=user_array[i], cutoff=10)
        #               else:
        #                    recommandations=self.Recommender.recomendation_ranking(user_id_array=user_array[i])
        array = np.asarray(recommandations)
        writer.writerow(array)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
IALSRecommender: Epoch 1 of 115. Elapsed time 15.49 sec
IALSRecommender: Epoch 2 of 115. Elapsed time 31.89 sec
IALSRecommender: Epoch 3 of 115. Elapsed time 47.32 sec
IALSRecommender: Epoch 4 of 115. Elapsed time 1.05 min
IALSRecommender: Epoch 5 of 115. Elapsed time 1.34 min
IALSRecommender: Epoch 6 of 115. Elapsed time 1.60 min
IALSRecommender: Epoch 7 of 115. Elapsed time 1.83 min
IALSRecommender: Epoch 8 of 115. Elapsed time 2.06 min
IALSRecommender: Epoch 9 of 115. Elapsed time 2.30 min
IALSRecommender: Epoch 10 of 115. Elapsed time 2.53 min
IALSRecommender: Epoch 11 of 115. Elapsed time 2.76 min
IALSRecommender: Epoch 12 of 115. Elapsed time 2.99 min
IALSRecommender: Epoch 13 of 115. Elapsed time 3.24 min
IALSRecommender: Epoch 14 of 115. Elapsed time 3.46 min
IALSRecommender: Epoch 15 of 115. Elapsed time 3.70 min
IALSRecommender: Epoch 16 of 115. Elapsed time 3.93 min
IALSRecommender: Epoch 1

/bin/sh: free: command not found


Processed 41629 (100.0%) in 1.23 sec. BPR loss is 3.64E-02. Sample per second: 33765
SLIM_BPR_Recommender: Epoch 1 of 140. Elapsed time 0.87 sec
Processed 41629 (100.0%) in 0.72 sec. BPR loss is 1.08E-01. Sample per second: 57992
SLIM_BPR_Recommender: Epoch 2 of 140. Elapsed time 1.36 sec
Processed 41629 (100.0%) in 0.98 sec. BPR loss is 1.73E-01. Sample per second: 42492
SLIM_BPR_Recommender: Epoch 3 of 140. Elapsed time 1.62 sec
Processed 41629 (100.0%) in 1.18 sec. BPR loss is 2.19E-01. Sample per second: 35389
SLIM_BPR_Recommender: Epoch 4 of 140. Elapsed time 1.81 sec
Processed 41629 (100.0%) in 0.36 sec. BPR loss is 2.86E-01. Sample per second: 114860
SLIM_BPR_Recommender: Epoch 5 of 140. Elapsed time 2.00 sec
Processed 41629 (100.0%) in 0.56 sec. BPR loss is 3.19E-01. Sample per second: 74677
SLIM_BPR_Recommender: Epoch 6 of 140. Elapsed time 2.19 sec
Processed 41629 (100.0%) in 0.73 sec. BPR loss is 3.62E-01. Sample per second: 57172
SLIM_BPR_Recommender: Epoch 7 of 140. Elapse

In [5]:
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender

rp3=RP3betaRecommender(URM_train)
rp3.fit(
    topK= 339, alpha= 0.0, beta= 0.3326953675715214, normalize_similarity= True
)
ItemKNNCF = ItemKNNCFRecommender(URM_train)
ItemKNNCF.fit()

alpha=50
beta=30

higher_recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_train,
    recommenders=[
        ItemKNNCF,
        slimBPR,
        rp3

    ],
    verbose=False
)
higher_recommender.fit(
    alphas=[
        alpha,beta,
        1 - alpha - beta
    ]
)
result_df, _ = evaluator_test.evaluateRecommender(higher_recommender)
result_df

RP3betaRecommender: Similarity column 24507 (100.0%), 1319.58 column/sec. Elapsed time 18.57 sec
Similarity column 24507 (100.0%), 4694.05 column/sec. Elapsed time 5.22 sec


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 41301 (100.0%) in 1.20 min. Users per second: 574


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.056529,0.076436,0.066035,0.028694,0.038034,0.181033,NaN,0.060913,0.355028,0.225601,...,0.992121,0.35223,0.992121,0.035549,9.035067,0.991614,0.070045,0.669578,4.200665,0.268681


In [ ]:
alpha = 0.7
new_similarity = (1 - alpha) * ItemKNNCF.W_sparse + alpha * P3alpha.W_sparse # Weighted Sum
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)

result_df, _ = evaluator_test.evaluateRecommender(recommender_object)
result_df

In [ ]:
from Recommenders.DifferentLossScoresHybridRecommender import DifferentLossScoresHybridRecommender
import numpy as np

recommender_object = DifferentLossScoresHybridRecommender(URM_train, ials, slimBPR)

for norm in [1, 2, np.inf, -np.inf]:

    recommender_object.fit(norm, alpha = 0.66)

    result_df, _ = evaluator_test.evaluateRecommender(recommender_object)
    print("Norm: {}, Result: {}".format(norm, result_df.loc[10]["MAP"]))

In [3]:
import numpy as np
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import scipy.sparse as sps
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
ICM=Reader.load_ICM_type()
MAP_recommender_per_group = {}
'''
collaborative_recommender_class = {"TopPop": TopPop,
                                   "UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "PureSVD": PureSVDRecommender,
                                   "NMF": NMFRecommender,
                                   "FunkSVD": MatrixFactorization_FunkSVD_Cython,
                                   "SLIMBPR": SLIM_BPR_Cython,
                                   }

content_recommender_class = {"ItemKNNCBF": ItemKNNCBFRecommender,
                             "ItemKNNCFCBF": ItemKNN_CFCBF_Hybrid_Recommender
                             }
'''

collaborative_recommender_class = {
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "PureSVD": PureSVDRecommender,
                                   "SLIMBPR": SLIM_BPR_Cython,
                                   }



recommender_object_dict = {}
profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)
sorted_users = np.argsort(profile_length)
block_size = int(len(profile_length)*0.05)

for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train)
    recommender_object.fit()
    recommender_object_dict[label] = recommender_object

cutoff = 10


for group_id in range(0, 20):

    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]

    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id,
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))


    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]

    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

    for label, recommender in recommender_object_dict.items():
        result_df, _ = evaluator_test.evaluateRecommender(recommender)
        if label in MAP_recommender_per_group:
            MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
        else:
            MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]


Deallocating Cython objects
Similarity column 24507 (100.0%), 4224.52 column/sec. Elapsed time 5.80 sec
P3alphaRecommender: Similarity column 24507 (100.0%), 893.91 column/sec. Elapsed time 27.41 sec
RP3betaRecommender: Similarity column 24507 (100.0%), 746.19 column/sec. Elapsed time 32.84 sec
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.47 sec
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.


/bin/sh: free: command not found


Processed 41629 (100.0%) in 3.10 sec. BPR loss is 2.08E-02. Sample per second: 13446
SLIM_BPR_Recommender: Epoch 1 of 140. Elapsed time 2.21 sec
Processed 41629 (100.0%) in 1.40 sec. BPR loss is 5.79E-02. Sample per second: 29729
SLIM_BPR_Recommender: Epoch 2 of 140. Elapsed time 3.50 sec
Processed 41629 (100.0%) in 1.59 sec. BPR loss is 8.90E-02. Sample per second: 26249
SLIM_BPR_Recommender: Epoch 3 of 140. Elapsed time 4.69 sec
Processed 41629 (100.0%) in 1.77 sec. BPR loss is 1.19E-01. Sample per second: 23475
SLIM_BPR_Recommender: Epoch 4 of 140. Elapsed time 5.88 sec
Processed 41629 (100.0%) in 1.89 sec. BPR loss is 1.46E-01. Sample per second: 22046
SLIM_BPR_Recommender: Epoch 5 of 140. Elapsed time 6.99 sec
Processed 41629 (100.0%) in 1.95 sec. BPR loss is 1.67E-01. Sample per second: 21311
SLIM_BPR_Recommender: Epoch 6 of 140. Elapsed time 8.05 sec
Processed 41629 (100.0%) in 1.86 sec. BPR loss is 2.03E-01. Sample per second: 22377
SLIM_BPR_Recommender: Epoch 7 of 140. Elapsed

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 2047 (100.0%) in 3.73 sec. Users per second: 549


...

EvaluatorHoldout: Processed 2081 (100.0%) in 4.33 sec. Users per second: 480


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 2081 (100.0%) in 3.53 sec. Users per second: 589


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 2081 (100.0%) in 3.41 sec. Users per second: 610
Group 19, #users in group 2081, average p.len 105.80, median 89.0, min 69, max 457
EvaluatorHoldout: Ignoring 311 ( 0.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 39548 Users
EvaluatorHoldout: Processed 2081 (100.0%) in 5.48 sec. Users per second: 380
EvaluatorHoldout: Processed 2081 (100.0%) in 3.44 sec. Users per second: 605
EvaluatorHoldout: Processed 2081 (100.0%) in 4.38 sec. Users per second: 475
EvaluatorHoldout: Processed 2081 (100.0%) in 3.03 sec. Users per second: 688
EvaluatorHoldout: Processed 2081 (100.0%) in 3.36 sec. Users per second: 620


In [4]:
MAP_recommender_per_group

{'ItemKNNCF': [0.003428610237901187,
  0.005539005032907467,
  0.0062591262090773755,
  0.007117345554082235,
  0.008232493679400644,
  0.010225653390210374,
  0.010551240560949342,
  0.01322620401971944,
  0.015030650678350502,
  0.016733836530249572,
  0.02062626646403248,
  0.02397317598210461,
  0.02523382850407923,
  0.032520156210632443,
  0.037387929320732725,
  0.04380744429181921,
  0.04921262289955221,
  0.05759921588369443,
  0.0723737824458631,
  0.10104677238507134],
 'P3alpha': [0.004601932704622184,
  0.005991889276035612,
  0.007444099547371026,
  0.00839934166143253,
  0.009284612529863404,
  0.010918977295559603,
  0.011593446601941783,
  0.01262744460696683,
  0.015696587109420127,
  0.015178042456278094,
  0.017758377851462996,
  0.021487693050193102,
  0.02330287306179306,
  0.02670938409033647,
  0.03199993500932809,
  0.036430307539682516,
  0.04158596675896048,
  0.04610014644973802,
  0.05663161788822527,
  0.07625302243274373],
 'RP3beta': [0.00453523887142468

In [ ]:
algoList=[pureSVD,P3alpha,rp3]

best=0
alpha=0
algo=""
dList=numpy.arange(0.25, 1, 0.25)
for x in tqdm(itertools.product(dList,algoList),total=int(len(dList)*len(algoList)), position=0, leave=True):

    scoreshybridrecommender = ScoresHybridRecommender(URM_train, ials, x[1])
    scoreshybridrecommender.fit(alpha = x[0])
    result_df, _ = evaluator_test.evaluateRecommender(scoreshybridrecommender)


    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        alpha=x[0]
        algo=x[1]

best,alpha,algo